In [1]:
from __future__  import print_function  # Python 2/3 compatibility

import pkg_resources
import json
import pandas as pd
import boto3
pkg_resources.get_distribution("boto3").version

'1.19.3'

In [2]:
# Need to update to crawl S3 for src folders 
s3 = boto3.client('s3')

response = s3.get_object(
    Bucket='bearly-graph',
    Key='src/mpd.slice.0-999.json',
)
# Extracts body of the 
body = response['Body']
str_body = body.read()
s3_obj_content = json.loads(str_body)
playlists = s3_obj_content['playlists']

In [3]:
album_list = []

for playlist in playlists:
    tracks = playlist['tracks']
    for track in tracks:
        album_dict = {}
        album_dict['~id'] = track['album_uri']
        album_dict['album_name:String'] =  track['album_name']
        album_dict['label'] =  'album'
        album_list.append(album_dict) 

df = pd.DataFrame.from_dict(album_list)
df.drop_duplicates()
b_string = df.to_csv().encode('utf-8')
key = f'vertex/version=2/album_vertex_{i}.csv'
s3.put_object(Body=b_string, Bucket='bearly-graph', Key=key)